<a href="https://colab.research.google.com/github/VasundharaAgarwal/GANFingerprinting/blob/main/Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
import os
import random
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torchvision
import pandas as pd
import torchvision.utils as vutils
import torch.nn as nn

import torch.nn.functional as F

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import sys
sys.path.append('/content/gdrive/My Drive/')
import model_architectures as ma

In [8]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        
        #encoder 
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 4, 3, padding=1)
        self.maxPool = nn.MaxPool2d(2, 2)  
       
        # decoder 
        self.convTr1 = nn.ConvTranspose2d(4, 16, 2, stride=2)
        self.convTr2 = nn.ConvTranspose2d(16, 1, 2, stride=2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.maxPool(x)
        x = F.relu(self.conv2(x))
        x = self.maxPool(x)  
        x = F.relu(self.convTr1(x))
        x = F.sigmoid(self.convTr2(x))
                
        return x



In [9]:
autoEnc = Autoencoder()
print(autoEnc)

Autoencoder(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxPool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (convTr1): ConvTranspose2d(4, 16, kernel_size=(2, 2), stride=(2, 2))
  (convTr2): ConvTranspose2d(16, 3, kernel_size=(2, 2), stride=(2, 2))
)


In [11]:
x = torch.randn(5,2)
x[0]

tensor([ 0.7754, -0.9387])

In [12]:
real_fp = torch.randn(784, requires_grad=True)
gan1_fp = torch.randn(784, requires_grad=True)
gan2_fp = torch.randn(784, requires_grad=True)
gan3_fp = torch.randn(784, requires_grad=True)
fps = [real_fp, gan1_fp, gan2_fp, gan3_fp]

<function Tensor.type>

In [32]:
def custom_loss(output, target, source):
  l1Criterion = nn.L1Loss()
  l1_loss = l1Criterion(output, target)
  cum_corr_loss = torch.zeros(1)
  for i, recon_img in enumerate(output):
    target_img = target[i]
    img_fp = recon_img - target_img
    img_fp_flat = torch.flatten(img_fp)
    vimg_fp_flat = img_fp_flat - torch.mean(img_fp_flat)
    sum = 0
    numerator = 0
    for i, fp in enumerate(fps):
      vx = vimg_fp_flat
      vy = fp - torch.mean(fp)
      cost = torch.sum(vx * vy) / (torch.sqrt(torch.sum(vx ** 2)) * torch.sqrt(torch.sum(vy ** 2)))
      if(i == source):
        numerator = cost
      sum+=cost
    ind_corr_loss = -torch.log(numerator/sum)
    print("num = ",numerator)
    print("sum =", sum)
    print("log = ", ind_corr_loss)
    cum_corr_loss += ind_corr_loss
  return torch.mean(cum_corr_loss) + l1_loss




In [33]:
img = torch.randn(4,28,28)
custom_loss(img, img*2, 0)

num =  tensor(-0.0027, grad_fn=<DivBackward0>)
sum = tensor(0.0323, grad_fn=<AddBackward0>)
log =  tensor(nan, grad_fn=<NegBackward>)
num =  tensor(-0.0149, grad_fn=<DivBackward0>)
sum = tensor(0.0772, grad_fn=<AddBackward0>)
log =  tensor(nan, grad_fn=<NegBackward>)
num =  tensor(0.0114, grad_fn=<DivBackward0>)
sum = tensor(-0.0818, grad_fn=<AddBackward0>)
log =  tensor(nan, grad_fn=<NegBackward>)
num =  tensor(-0.0037, grad_fn=<DivBackward0>)
sum = tensor(-0.0118, grad_fn=<AddBackward0>)
log =  tensor(1.1560, grad_fn=<NegBackward>)


tensor(nan, grad_fn=<AddBackward0>)

In [ ]:
optimizer = optim.Adam(autoEnc.parameters())

In [ ]:
def trainGAN(num, netG, netD):
  netG = netG.to(device)
  netD = netD.to(device)
  optimD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
  optimG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))
  img_list = []
  D_x_list = []
  D_G_z1_list = []
  D_G_z2_list = []
  iters = 0
  for epoch in range(num_epochs):
      print(epoch)
      # For each batch in the dataloader
      for i, data in enumerate(train_loader, 0):

          ############################
          # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
          ###########################
          ## Train with all-real batch
          netD.zero_grad()
          # Format batch
          real_cpu = data[0].to(device)
          b_size = real_cpu.size(0)
          label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
          # Forward pass real batch through D
          output = netD(real_cpu).view(-1)
          # Calculate loss on all-real batch
          errD_real = criterion(output, label)
          # Calculate gradients for D in backward pass
          errD_real.backward()
          D_x = output.mean().item()

          ## Train with all-fake batch
          # Generate batch of latent vectors
          noise = torch.randn(b_size, ma.nz, 1, 1, device=device)
          # Generate fake image batch with G
          fake = netG(noise)
          label.fill_(fake_label)
          # Classify all fake batch with D
          output = netD(fake.detach()).view(-1)
          # Calculate D's loss on the all-fake batch
          errD_fake = criterion(output, label)
          # Calculate the gradients for this batch
          errD_fake.backward()
          D_G_z1 = output.mean().item()
          # Add the gradients from the all-real and all-fake batches
          errD = errD_real + errD_fake
          # Update D
          optimD.step()

          ############################
          # (2) Update G network: maximize log(D(G(z)))
          ###########################
          netG.zero_grad()
          label.fill_(real_label)  # fake labels are real for generator cost
          # Since we just updated D, perform another forward pass of all-fake batch through D
          output = netD(fake).view(-1)
          # Calculate G's loss based on this output
          errG = criterion(output, label)
          # Calculate gradients for G
          errG.backward()
          D_G_z2 = output.mean().item()
          # Update G
          optimG.step()

          # Output training stats
          if i % 50 == 0:
              print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                    % (epoch, num_epochs, i, len(train_loader),
                      errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

          # Save Losses for plotting later
          D_x_list.append(D_x)
          D_G_z1_list.append(D_G_z1)
          D_G_z2_list.append(D_G_z2)
          # Check how the generator is doing by saving G's output on fixed_noise
          if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(train_loader)-1)):
              with torch.no_grad():
                  fake = netG(fixed_noise).detach().cpu()
              img_list.append(vutils.make_grid(fake, padding=2, normalize=True))
          iters += 1
